In [1]:
import numpy as np
import pandas as pd
import spiceypy as spice
from astroquery.jplhorizons import Horizons
import astropy.units as u
from astropy.constants import c
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.time import Time

In [2]:
def Furnisher(k):
    '''
    This function is used to load all kernels needed in an operation.
    Comment out kernels not in use and add the ones in use.
    
    Arguments: NA
    Returns: NA
    
    '''
    spice.kclear()
    spice.furnsh('/Users/user/Downloads/naif0009.tls.txt')
    if k == '310+341+435':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    elif k == '310+341':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
    elif k == '310+435':
            spice.furnsh('/Users/user/Downloads/de435.bsp')
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
    elif k == '341+435':
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    elif k == '310':
            spice.furnsh('/Users/user/Downloads/jup310.bsp')
    elif k == '341':
            spice.furnsh('/Users/user/Downloads/jup341.bsp')
    elif k == '435':
            spice.furnsh('/Users/user/Downloads/de435.bsp')
    pass

In [3]:
Furnisher("310+341+435")

In [4]:
def monthToNum(shortMonth):
    
    '''
    
    Returns the name of a month that a number corresponds to and vice versa
    
    Arguments:
    
    shortMonth: str
    
    Either the first three letters of a month or its number (e.g. 01 for January, 10 for October)
    
    Returns:
    
    shortMonth: str
    
    Either the first three letters of a month or its number (e.g. 01 for January, 10 for October),
    the other one to the one that was given as an argument
    
    '''
    
    return{
        'Jan' : '01', 'Feb' : '02','Mar' : '03','Apr' : '04','May' : '05','Jun' : '06','Jul' : '07','Aug' : '08',
        'Sep' : '09', 'Oct' : '10','Nov' : '11','Dec' : '12','01' : 'Jan','02' : 'Feb','03' : 'Mar','04' : 'Apr',
        '05' : 'May','06' : 'Jun','07' : 'Jul','08' : 'Aug','09' : 'Sep', '10' : 'Oct','11' : 'Nov','12' : 'Dec'}[shortMonth]

In [5]:
def JD_to_SpiceJD(jd):
    '''
    
    Converts jd date to spice units of time.
    
    Arguments:
    
    jd: float
    
    A jd time in days
    
    Returns:
    
    newjd: float
    
    A spice usable time in seconds
    
    
    '''
    t = Time(jd, format='jd', scale='utc')
    k = t.fits
    year = k[0:4]
    month = k[5:7]
    day = k[8:10]
    hours = k[11:13]
    minutes = k[14:16]
    seconds = k[17:23]
    newmnth = monthToNum(month)
    utc = newmnth + ' ' + day + ", " + year
    newjd = spice.str2et(utc) + int(hours)*3600 + int(minutes)*60 + float(seconds)
    return newjd

In [6]:
def get_spice_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are 'NONE', 'LT','LT+S'
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        
        radec: numpy array
        
        The right ascension and declination of the observed object
        """
        
        state,lighttime = spice.spkezr(name,t,'J2000',cor,loc)
        pos,lighttime = spice.spkpos(name,t,'J2000',cor,loc)
        range,ra,dec = spice.recrad(pos) 
        xyz = np.array([state[0],state[1],state[2]])/149597870.7#6.68459e-9
        uvw = np.array([state[3],state[4],state[5]])/149597870.7*24.*3600.#*6.68459e-9
        radec = np.array([ra,dec])
        return xyz,uvw,radec*180/np.pi
    return get_target_xyz

In [7]:
def get_astroquery_function(name,cor,loc):
    """
    This wrapper function automates the creation of objects through the JPL Horizons database. 
    
    Arguments:
    
    name: str
    
    Stipulates the target object in Horizons. The major bodies in the Solar System have an id based on their position.
    Hence '5' refers to Jupiter and '3' to Earth. A single number designator refers to a barycenter and a designator
    such as '599' to the planetary center. For minor bodies in the Solar System, the id_type in the Horizons
    must be changed to "minorbody"
    
    cor: str
    
    Refers to the type of correction that the object has. Available arguments are "geometric","astrometric" and 
    "apparent"
    
    loc: str
    
    Designates the location of observation. Names that start with "g@#" refer to barycenters where the number designates the 
    body that the observer is based at. Hence "g@0" refers to the Solar System barycenter. Also takes Earth location designators.
    Observatories are named after their code. Hence, Pan-Starrs observatory is referred as "f51"

    Returns:
    
    get_target_xyz function
    """    
    def get_target_xyz(t):
        """
        Returns the vectors of the Horizons body at a certain time t.
        
        Arguments:
        
        t: days
        
        Julian date of observation
        
        Returns:
    
        xyz: numpy array
        
        A position vector of the observed object
    
        uvw: numpy array
        
        An instantaneous velocity vector of the observed object
        """
        
        obj = Horizons(id = name, location = loc, epochs = t, id_type = 'majorbody')
        obj1 = obj.vectors(aberrations = cor, refplane = 'earth')
        xyz = np.array([float(obj1['x']),float(obj1['y']),float(obj1['z'])])
        uvw = np.array([float(obj1['vx']),float(obj1['vy']),float(obj1['vz'])])
        obj2 = obj.ephemerides(refsystem = 'J2000', extra_precision = True)
        radec = np.array([float(obj2['RA']),float(obj2['DEC']),float(obj1['range'])])
        return xyz,uvw,radec
    return get_target_xyz    

In [8]:
def strip(x):
    temp = x.strip(" [")
    return temp.strip("]")
names = ['ID', 'smf-name', 'smf-filepath', 'reduction', 'JDUTC', 'RA', 'Dec', 'filter', 'exp-time', 'UVx', 'UVy', 'UVz', 'obseqx', 'obseqy', 'obseqz', 'obseclx', 'obseclsy', 'obseclz']
df = pd.read_csv("/Users/user/Irregular-Satellites/amalgamated_unique_exposure_reads.txt", names = names, converters={col : strip for col in ['UVx', 'UVz', 'obseqx', 'obseqz', 'obseclx', 'obseclz']})

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
def ImageFinder(df,func,i):
    try: 
        time = float(df["JDUTC"][i])
        ts = JD_to_SpiceJD(time)
        xyz,uvw,astrRD = func(ts)
        RADEC = np.array([float(df["RA"][i]),float(df["Dec"][i])])
        HillR = 0.177141559/np.linalg.norm(xyz)
        if (RADEC[0]-(3.2+HillR) <= astrRD[0] <= RADEC[0]+(3.2+HillR)) and (RADEC[1]-(3.2+HillR) <= astrRD[1] <= RADEC[1]+(3.2+HillR)):
            return True,astrRD,RADEC,time
        else:
            return False,astrRD,RADEC,time
    except:
            return None,None,None,None
        
        

Takes a while to load, it's close to a million results

In [18]:
get_jup = get_spice_function("5","NONE","3")
bools250k = []
QueryRADec250k = []
PANSTRADec250k = []
times250k = []

for i in range(0,250000):
    k = ImageFinder(df,get_jup,i)
    bools250k.append(k[0])
    times250k.append(k[3])
    QueryRADec250k.append(k[1])
    PANSTRADec250k.append(k[2])

In [18]:
get_jup = get_spice_function("5","NONE","3")
bools500k = []
QueryRADec500k = []
PANSTRADec500k = []
times500k = []

for i in range(250000,500000):
    k = ImageFinder(df,get_jup,i)
    bools500k.append(k[0])
    times500k.append(k[3])
    QueryRADec500k.append(k[1])
    PANSTRADec500k.append(k[2])

In [18]:
get_jup = get_spice_function("5","NONE","3")
bools750k = []
QueryRADec750k = []
PANSTRADec750k = []
times750k = []

for i in range(500000,750000):
    k = ImageFinder(df,get_jup,i)
    bools750k.append(k[0])
    times750k.append(k[3])
    QueryRADec750k.append(k[1])
    PANSTRADec750k.append(k[2])

In [18]:
get_jup = get_spice_function("5","NONE","3")
bools850k = []
QueryRADec850k = []
PANSTRADec850k = []
times850k = []

for i in range(750000,821774):
    k = ImageFinder(df,get_jup,i)
    bools850k.append(k[0])
    times850k.append(k[3])
    QueryRADec850k.append(k[1])
    PANSTRADec850k.append(k[2])

In [19]:
bools250k.count(True)

211

In [20]:
bools500k.count(True)

286

In [22]:
bools750k.count(True)

390

In [23]:
bools850k.count(True)

76